### Crawl data from lbar.com

In [147]:
# Many thanks to Son Nguyen

In [148]:
#collecting data about Reator on lbar.com

In [149]:
#!pip install pandoc -->for converting jupyter notebook to .pdf
#!pip install nbconvert[webpdf] -->for converting to .pdf

!pip install beautifulsoup4

import requests
from bs4 import BeautifulSoup
import re  #dùng cho tách số điện thoại

In [150]:
link_base ="https://www.lbar.com/index.php?src=directory&view=rets_flex_active_agents"\
"&srctype=rets_flex_active_agents_lister&xsearch_id=rets_flex_active_agents_alpha"\
"&xsearch=dummy&query=name.starts."

#print(link_base)

In [151]:
domain_base = "https://www.lbar.com/"

#### thêm thông số phân trang: &pos=0,2000,2000&xsearch_id=rets_flex_active_agents_alpha&xsearch=dummy

In [152]:
list_link =[]
for one in range(97,123):
    item = link_base + chr(one) + "&pos=0,2000,2000&xsearch_id=rets_flex_active_agents_alpha&xsearch=dummy"
    list_link.append(item)
    #print(list_link)

In [153]:
#print(len(list_link))

##### Phân tích cú pháp và bóc tách dữ liệu-->sử dụng Beautifulsoup

In [156]:
# headers là cần thiết để tránh trường hợp website xem là robot

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

f = open("lbar_realtor.csv", "a")  #tạo file lbar_realtor.csv để lưu dữ liệu 

head ="Link_profile, Realtor, Job, Address, Email, Phone number,\n" #thêm Header cho file lbar_realtor.csv
f.write(head)

for i in range(0,len(list_link)): #ránh mất dữ liệu khi lỗi mạng, muốn chạy tiếp tục chứ ko phải chạy từ đầu
    print("counter:", i) #in ra lần chạy
    
    link = list_link[i]
    r = requests.get(link, headers=headers) # headers là cần thiết để tránh trường hợp website xem là robot
    #print(r.text)
    soup = BeautifulSoup(r.text, 'html.parser')
    tb_data = soup.find_all('table') # lấy các table
    #soup.find('tbody')

    #print(tb_data[0].find('tbody')) # lấy phần tử đầu tiên tb_data[0] vì trang này chỉ có 1 table cũng chính là table cần thao tác
    tbody_data = tb_data[0].find('tbody')
    
    tr_data = soup.find_all('tr') # lấy ra tất cả dòng tr, có thể bỏ luôn 2 bước tb_data, tbody_data
    #print(tr_data) #  -->tr_data ở dạng mảng 
    
    #vì tr_data ở dạng mảng nên có thể dòng vòng lặp for với row
    for row in tr_data[1:]: #cột đầu (td) là header, không cần lấy nên chọn cột thứ 2 đến hết
        #print(row.find_all('td')) # lấy về từng cái td
        td_list = row.find_all('td')
        #td_list[0] -->phần tử đầu tiên chứa href
        
        link_profile = td_list[0].find_all("a")[0].get('href') #thẻ a thứ nhất a[0]
        link_p = domain_base + link_profile
        #print(link_p)
        
        name = td_list[1].text
        names = name.split("\n")
        names.remove('') #xóa ký tự rỗng
        
        name1 = names[0].strip().replace(",", "-") # cắt khoảng trống 2 đầu ký tự thứ nhất
        name2 = names[1].strip().replace(",", "-")
        #print(name2) 
        
        # địa chỉ chỉ 1 dòng nên cần td_list[2].text, dù có xuống dòng 
        addr = td_list[2].text.strip().replace(",", "-").replace("\n", '') #replace("\n", ''): xóa ký tự xuống dòng
        #print(addr)
        
        #phone number and e-mail
        #contact_data = td_list[3].text.strip()
        #print(contact_data)
        
        #email
        try:
            email = td_list[3].find_all("a")[0].text
            #print(email)
        except:
            email ='' #tránh báo lỗi trong trường hợp không có email
        
        #phone number
        try:
            #regex ="\([0-9]{3}\)\s+[0-9]{3}\-[0-9]{4}"
            phone = td_list[3].text
            p = re.compile(r"\([0-9]{3}\)\s+[0-9]{3}\-[0-9]{4}")
            x = p.search(phone)
            pnb = str(x.group(0))
            #print(pnb)
        except:
            pnb ='' #tránh báo lỗi trong trường hợp không có phone number

        str1 =f"{link_p},{name1},{name2},{addr}, {email}, {pnb}, \n" #tạo chuỗi chứa nội dung, để lưu thành file .csv (f= format)
        
        f.write(str1)
        
        
f.close()



counter: 0
counter: 1
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
